# Data check

[Simon Dobson](mailto:simon.dobson@st-andrews.ac.uk) <br>
School of Computer Science, University of St Andrews, Scotland UK

This notebook is intended to check the rain gauge datasets downloaded and created from SEPA and CEDA. 

In [1]:
import json
import netCDF4 as nc
from geopandas import GeoDataFrame

from pyproj import CRS, Transformer
import folium

In [8]:
uk_grid_crs = CRS.from_string('EPSG:27700')   # UK national grid
latlon_crs = CRS.from_string('EPSG:4326')     # global Mercator (WGS 84)

proj = Transformer.from_crs(uk_grid_crs, latlon_crs)

## County boundaries

In [2]:
boundaries_filename = 'datasets/UK_BUC.geojson'
with open(boundaries_filename, 'r') as fh:
    counties_json = json.load(fh)

In [3]:
counties = GeoDataFrame(columns=['county', 'geometry'])
for c in counties_json['features']:
    counties.loc[len(counties.index)] = {'county': c['properties']['ctyua18nm'],
                                         'geometry': shape(c['geometry'])}

## SEPA rain gauges and a sample month

In [39]:
sepa_filename='datasets/sepa_monthly_2017.nc'
sepa = nc.Dataset(sepa_filename)

In [40]:
sepastations = GeoDataFrame(columns=['name', 'id', 'longitude', 'latitude', 'geometry'])
for i in range(len(sepa['station'])):
    lat, lon = proj.transform(float(sepa['x'][i]), float(sepa['y'][i]))
    sepastations.loc[i] = {'id': int(sepa['station'][i]),
                           'name': sepa['name'][i],
                           'longitude': lon,
                           'latitude': lat,
                           'geometry': Point(lon, lat)}

In [41]:
sepastations

,name,id,longitude,latitude,geometry
0,Abbey St Bathans,15018,-2.400869,55.850549,POINT (-2.40087 55.85055)
1,Aberlour,234150,-3.218772,57.471363,POINT (-3.21877 57.47136)
2,Affric Lodge,115301,-5.019301,57.261328,POINT (-5.01930 57.26133)
3,Alford,234170,-2.730662,57.241578,POINT (-2.73066 57.24158)
4,Allanfearn,115302,-4.153978,57.494793,POINT (-4.15398 57.49479)
...,...,...,...,...,...
272,Waulkmill Glen,115660,-4.362137,55.792568,POINT (-4.36214 55.79257)
273,Weisdale Mill,116008,-1.297096,60.259372,POINT (-1.29710 60.25937)
274,Westhill,115250,-2.299088,57.144534,POINT (-2.29909 57.14453)
275,Whitburn,14881,-3.695373,55.866478,POINT (-3.69537 55.86648)


## CEDA rain gauges and a sample month 

In [54]:
ceda_filename='datasets/ceda_monthly_2017.nc'
ceda = nc.Dataset(ceda_filename)

In [55]:
cedastations = GeoDataFrame(columns=['name', 'id', 'longitude', 'latitude', 'geometry'])
for i in range(len(ceda['station'])):
    lat, lon = proj.transform(float(ceda['x'][i]), float(ceda['y'][i]))
    cedastations.loc[i] = {'id': int(ceda['station'][i]),
                           'name': ceda['name'][i],
                           'longitude': lon,
                           'latitude': lat,
                           'geometry': Point(lon, lat)}

In [61]:
cedastations

,name,id,longitude,latitude,geometry
0,achfary,38,-4.920076,58.306182,POINT (-4.92008 58.30618)
1,huntsham,8231,-3.439265,50.952553,POINT (-3.43927 50.95255)
2,strathy-east,50,-3.995386,58.557531,POINT (-3.99539 58.55753)
3,poolewe,60,-5.599708,57.767447,POINT (-5.59971 57.76745)
4,plockton,64,-5.656926,57.334277,POINT (-5.65693 57.33428)
...,...,...,...,...,...
146,mannington-hall,24219,1.176164,52.842971,POINT (1.17616 52.84297)
147,bute-rothesay-no2,57118,-5.066457,55.821956,POINT (-5.06646 55.82196)
148,margam-no-2,57233,-3.732128,51.551088,POINT (-3.73213 51.55109)
149,saltfleetby-st-clements,57266,0.179242,53.395922,POINT (0.17924 53.39592)


## The dataset

In [72]:
uk_gauges = folium.Map(location=(55, -3), tiles="Stamen Terrain", zoom_start=6)

# add the boundaries
county_boundaries_layer = folium.FeatureGroup(name='County boundaries')
for _, r in counties.iterrows():
    folium.GeoJson(r['geometry']).add_to(county_boundaries_layer)
county_boundaries_layer.add_to(uk_gauges)
    
# add the SEPA stations
sepa_layer = folium.FeatureGroup(name='SEPA stations')
for i in range(len(sepastations)):
    s = sepastations.iloc[i]
    name, lon, lat = s['name'], s['longitude'], s['latitude']
    folium.Marker(location=(lat, lon),
                  tooltip=f'{name} ({lat:.2f}N, {lon:.2f}W)',
                  icon=folium.Icon(color='blue', icon='cloud')).add_to(sepa_layer)
sepa_layer.add_to(uk_gauges)

# add the CEDA stations
ceda_layer = folium.FeatureGroup(name='CEDA MIDAS stations')
for i in range(len(cedastations)):
    s = cedastations.iloc[i]
    name, lon, lat = s['name'], s['longitude'], s['latitude']
    folium.Marker(location=(lat, lon),
                  tooltip=f'{name} ({lat:.2f}N, {lon:.2f}W)',
                  icon=folium.Icon(color='green', icon='cloud')).add_to(ceda_layer)
ceda_layer.add_to(uk_gauges)
    
# add a layer countrol
_ = folium.LayerControl().add_to(uk_gauges)

In [73]:
uk_gauges